## Day 35 Lecture 2 Assignment

In this assignment, we will combine what we have learned so far about classification algorithms this week.

In [1]:
!pip install dask[dataframe] --upgrade --quiet
!pip install dask-ml[complete] --quiet
!pip install aiohttp --quiet
!pip install joblib --quiet
!pip install dask distributed --upgrade --quiet
!pip install scikit-learn==0.23.2 --quiet

In [2]:
import dask.dataframe as dd
import dask.array as da
from dask.distributed import Client, progress
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate, GridSearchCV
from sklearn.metrics import roc_auc_score
import joblib
from dask_ml.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, precision_score, recall_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from datetime import datetime
import numpy as np

import matplotlib.pyplot as plt

%matplotlib inline

In [3]:
client = Client(n_workers=6, threads_per_worker=12, memory_limit='4GB')
client

Client Scheduler: tcp://127.0.0.1:44425 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 6 Cores: 72 Memory: 24.00 GB


In [4]:
admission = dd.read_csv('https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/Admission_Predict.csv')

In [5]:
admission.head()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,324,107,4,4.0,4.5,8.87,1,0.76
2,3,316,104,3,3.0,3.5,8.00,1,0.72
3,4,322,110,3,3.5,2.5,8.67,1,0.80
4,5,314,103,2,2.0,3.0,8.21,0,0.65


In this assignment, we will predict the probability of a student being admitted to a PhD program given their stats. To make the predictions, find the median for the Chance of Admit column. Create an admit column where all probabilities above the median will receive a 1 for that column and all probabilities below the median will be a zero.

Below you will process and clean the data, try the SVM classifier, the gradient boosted decision tree classifier and XGBoost, and compare your results.

Have fun!

In [6]:
# Answer below:
admission.describe().compute()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
count,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000
mean,200.500000,316.807500,107.410000,3.087500,3.400000,3.452500,8.598925,0.547500,0.724350
std,115.614301,11.473646,6.069514,1.143728,1.006869,0.898478,0.596317,0.498362,0.142609
min,1.000000,290.000000,92.000000,1.000000,1.000000,1.000000,6.800000,0.000000,0.340000
25%,100.750000,308.000000,103.000000,2.000000,2.500000,3.000000,8.170000,0.000000,0.640000
50%,200.500000,317.000000,107.000000,3.000000,3.500000,3.500000,8.610000,1.000000,0.730000
75%,300.250000,325.000000,112.000000,4.000000,4.000000,4.000000,9.062500,1.000000,0.830000
max,400.000000,340.000000,120.000000,5.000000,5.000000,5.000000,9.920000,1.000000,0.970000


In [7]:
admission.columns

Index(['Serial No.', 'GRE Score', 'TOEFL Score', 'University Rating', 'SOP',
       'LOR ', 'CGPA', 'Research', 'Chance of Admit '],
      dtype='object')

In [8]:
new_columns = ['Serial No.', 'GRE Score', 'TOEFL Score', 'University Rating', 'SOP',
       'LOR', 'CGPA', 'Research', 'Chance of Admit']

admission.columns = new_columns

admission.columns

Index(['Serial No.', 'GRE Score', 'TOEFL Score', 'University Rating', 'SOP',
       'LOR', 'CGPA', 'Research', 'Chance of Admit'],
      dtype='object')

In [9]:
median = admission['Chance of Admit'].compute().median()

In [10]:
#admission['Chance of Admit'] = np.where(admission['Chance of Admit'] >= 0.73,1,0) ### np way

admission['Chance of Admit'] = admission['Chance of Admit'].map(lambda x: 1 if x >= median else 0)
admission['Chance of Admit'].value_counts().compute()

1    204
0    196
Name: Chance of Admit, dtype: int64

In [11]:
X = admission.drop(columns='Chance of Admit')
y = admission['Chance of Admit']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True)

In [12]:
#svm model
start_time = datetime.now()

svc = SVC(kernel='linear',C=5)

with joblib.parallel_backend('dask'):
  svc.fit(X_train.compute(), y_train.compute())

end_time = datetime.now()

print("Train score: ",svc.score(X_train.compute(), y_train.compute()))
print("Test score: ",svc.score(X_test.compute(), y_test.compute()))
print('Duration: {}'.format(end_time - start_time))

Train score:  0.8821752265861027
Test score:  0.8695652173913043
Duration: 0:00:02.023778


In [13]:
start_time = datetime.now()
param_grid = {'kernel': ['linear','rbf','poly'], 'C':np.arange(1,100,3)}

svc = SVC()

svc_grid = GridSearchCV(svc, 
                        param_grid=param_grid, 
                        scoring='accuracy', 
                        return_train_score=True, 
                        cv=3)

with joblib.parallel_backend('dask'):
  svc_grid.fit(X_train.compute(), y_train.compute())

end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))

Duration: 0:03:27.832104


In [14]:
print("Best kernal SVC hyperparameter: ",svc_grid.best_estimator_.kernel)
print("Best C SVC hyperparameter: ",svc_grid.best_estimator_.C)
print('------------------------------------------------')
print("Best Hyperparameters SVC Train score: ",svc_grid.score(X_train.compute(), y_train.compute()))
print("Best Hyperparameters SVC Test score: ",svc_grid.score(X_test.compute(), y_test.compute()))

Best kernal SVC hyperparameter:  linear
Best C SVC hyperparameter:  73
------------------------------------------------
Best Hyperparameters SVC Train score:  0.8761329305135952
Best Hyperparameters SVC Test score:  0.8840579710144928


In [15]:
start_time = datetime.now()
gbc = GradientBoostingClassifier()

with joblib.parallel_backend('dask'):
  gbc.fit(X_train.compute(), y_train.compute())
  
end_time = datetime.now()

print("Train score: ",gbc.score(X_train.compute(), y_train.compute()))
print("Test score: ",gbc.score(X_test.compute(), y_test.compute()))
print('Duration: {}'.format(end_time - start_time))

Train score:  0.9879154078549849
Test score:  0.855072463768116
Duration: 0:00:00.339989


In [16]:
start_time = datetime.now()
param_grid = {'learning_rate': np.arange(.01,.3,.01),'n_iter_no_change': [10],'n_estimators':[1000], 'max_depth': [2,3,], 
              'max_features': ['sqrt',.75,1]}

gbc = GradientBoostingClassifier()

gbc_grid = GridSearchCV(gbc, 
                        param_grid=param_grid, 
                        scoring='accuracy', 
                        return_train_score=True, 
                        cv=5)

with joblib.parallel_backend('dask'):
  gbc_grid.fit(X_train.compute(), y_train.compute())
  
end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))

Duration: 0:00:27.177963


In [17]:
print("Best learning rate GBC hyperparameter: ",gbc_grid.best_estimator_.learning_rate)
print("Best max depth GBC hyperparameter: ",gbc_grid.best_estimator_.max_depth)
print("Best max features GBC hyperparameter: ",gbc_grid.best_estimator_.max_features)
print('------------------------------------------------')
print("Best Hyperparameters GBC Train score: ",gbc_grid.score(X_train, y_train))
print("Best Hyperparameters GBC Test score: ",gbc_grid.score(X_test, y_test))

Best learning rate GBC hyperparameter:  0.06999999999999999
Best max depth GBC hyperparameter:  2
Best max features GBC hyperparameter:  sqrt
------------------------------------------------
Best Hyperparameters GBC Train score:  0.8972809667673716
Best Hyperparameters GBC Test score:  0.855072463768116


In [18]:
start_time = datetime.now()

xgb = XGBClassifier()

with joblib.parallel_backend('dask'):
  xgb.fit(X_train.compute(), y_train.compute())

end_time = datetime.now()

print("Train score: ",xgb.score(X_train.compute(), y_train.compute()))
print("Test score: ",xgb.score(X_test.compute(), y_test.compute()))
print('Duration: {}'.format(end_time - start_time))

Train score:  0.9788519637462235
Test score:  0.855072463768116
Duration: 0:00:00.292008


In [19]:
start_time = datetime.now()
param_grid = {'subsample': [.5,.75,1], 
              'learning_rate': np.arange(.01,.1,.01), 
              'max_depth': np.arange(3,10,1), 
              'lambda': [.5,.7,.9,1],
              }

xgb = XGBClassifier()

xgb_grid = GridSearchCV(xgb, 
                        param_grid=param_grid, 
                        scoring='accuracy', 
                        return_train_score=True, 
                        cv=3)

with joblib.parallel_backend('dask'):
  xgb_grid.fit(X_train.compute(), y_train.compute())

end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))

Duration: 0:00:40.464569


In [20]:
print("Best sub sample XGBoost hyperparameter: ",xgb_grid.best_estimator_.subsample)
print("Best learning rate XGBoost hyperparameter: ",xgb_grid.best_estimator_.learning_rate)
print("Best max depth XGBoost hyperparameter: ",xgb_grid.best_estimator_.max_depth)
print("Best Lamda XGBoost hyperparameter: ",xgb_grid.best_estimator_.reg_lambda)
print('------------------------------------------------')
print("Best Hyperparameters XGBoost Train score: ",xgb_grid.score(X_train.compute(), y_train.compute()))
print("Best Hyperparameters XGBoost Test score: ",xgb_grid.score(X_test.compute(), y_test.compute()))


Best sub sample XGBoost hyperparameter:  0.5
Best learning rate XGBoost hyperparameter:  0.09
Best max depth XGBoost hyperparameter:  3
Best Lamda XGBoost hyperparameter:  1
------------------------------------------------
Best Hyperparameters XGBoost Train score:  0.9516616314199395
Best Hyperparameters XGBoost Test score:  0.855072463768116


The SVC model had the best optimized test accuray of any model. It also did the least overfitting. The XGBoost model overfit the most and had the lowest test score. 